In [1]:
import os
os.chdir('../')

In [2]:
from src.ADB import ADB 
from src.Webcam import Webcam 
from src.Audio import Audio 
from src.StreamManager import StreamManager 
from PIL import ImageTk as PIL_ImageTk
import tkinter as tk
import tkinter.messagebox
from datetime import datetime
import time
import threading

import cv2
import sys
from PyQt5.QtWidgets import QWidget, QLabel, QApplication, QVBoxLayout,QPushButton
from PyQt5.QtCore import QThread, Qt, pyqtSignal, pyqtSlot
from PyQt5.QtGui import QImage, QPixmap

def init_stream_manager():
    sm=StreamManager()
    #sm.register_stream(stream_class=ADB,stream_type='video',stream_name='ADB')
    sm.register_stream(stream_class=Webcam,stream_type='video',device_id=0,stream_name='Webcam-1')
    sm.register_stream(stream_class=Audio,stream_type='audio',stream_name='Microphone',bitrate=44100,frames_per_buffer=1024,channels=2)
    sm.set_video_record_size('Webcam-1',(480,320))
    #sm.set_video_record_size('ADB',(295,640))
    sm.start()
    return sm

sm=init_stream_manager()

import cv2
import sys
from PyQt5.QtWidgets import QWidget, QLabel, QApplication, QVBoxLayout, QHBoxLayout,QMessageBox,QPushButton
from PyQt5.QtCore import QThread, Qt, pyqtSignal, pyqtSlot
from PyQt5.QtGui import QImage, QPixmap

class Thread(QThread):
    changePixmap = pyqtSignal(QImage)
    def set_params(self,sm,width,height):
        self.sm=sm
        self.width=width
        self.height=height
    def run(self):
        while self.is_running:
            frame = self.sm.get_current_frame(stream='Webcam-1',frame_size=(self.width,self.height))
            if frame is not None:
                rgbimage = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                h, w, ch = rgbimage.shape
                bytesPerLine = ch * w
                convertToQtFormat = QImage(rgbimage.data, w, h, bytesPerLine, QImage.Format_RGB888)
                image=convertToQtFormat.scaled(self.width,self.height,Qt.KeepAspectRatio) 
                self.changePixmap.emit(image)
            time.sleep(0.03)
                
class MyApp(QWidget):
    def __init__(self,app,sm):
        super(MyApp, self).__init__()
        self.title = 'Camera'
        self.is_running=True
        self.is_recording=False
        self.sm=sm
        self.initUI(app)

    def closeEvent(self,event):
        print('On close',flush=True)
        self.th0.is_running=False
        self.th1.is_running=False
        self.th2.is_running=False
        self.th0.quit()
        self.th1.quit()
        self.th2.quit()
        self.th0.wait()
        self.th1.wait()
        self.th2.wait()
        event.accept()
        print('Window closed')
     
    def button_on_click(self):
        if self.is_recording==False:
            self.is_recording=True
            self.button.setEnabled(False)
            session_name=datetime.now().strftime('%Y%m%d_%H%M%S')
            self.sm.start_recording(session_name)
            self.button.setText(f'End recording - {session_name}')
            self.button.setEnabled(True)
        else:
            self.is_recording=False
            self.button.setEnabled(False)
            self.button.setText('Processing, please wait')
            self.sm.stop_recording()
            self.button.setText('Processing complete, now you can quit')
    
    def initUI(self,app):
        top_layout=QVBoxLayout()
        button_panel=QHBoxLayout()
        window_layout=QHBoxLayout()
        
        self.button = QPushButton('Start record')
        self.button.clicked.connect(self.button_on_click)
        button_panel.addWidget(self.button)
        
        left = QVBoxLayout()
        self.label0 = QLabel(self)
        left.addWidget(self.label0)
        
        right = QVBoxLayout()
        self.label1 = QLabel(self)
        right.addWidget(self.label1)
        self.label2 = QLabel(self)
        right.addWidget(self.label2)  
        
        window_layout.addLayout(left)
        window_layout.addLayout(right)
        
        top_layout.addLayout(button_panel)
        top_layout.addLayout(window_layout)
        self.setLayout(top_layout)

        self.th0 = Thread()
        self.th0.set_params(self.sm,295,640)
        self.th0.changePixmap.connect(self.setImage0)
        self.th1 = Thread()
        self.th1.set_params(self.sm,480,320)
        self.th1.changePixmap.connect(self.setImage1)
        self.th2 = Thread()
        self.th2.set_params(self.sm,480,320)
        self.th2.changePixmap.connect(self.setImage2)
        self.th0.is_running=True
        self.th1.is_running=True
        self.th2.is_running=True
        #app.aboutToQuit.connect(self.close_event)
        self.th0.start()
        self.th1.start()
        self.th2.start()
        self.show()

    @pyqtSlot(QImage)
    def setImage0(self, image):
        self.label0.setPixmap(QPixmap.fromImage(image))
    def setImage1(self, image):
        self.label1.setPixmap(QPixmap.fromImage(image))
    def setImage2(self, image):
        self.label2.setPixmap(QPixmap.fromImage(image))

app = QApplication(sys.argv)
ex = MyApp(app,sm)

sys.exit(app.exec_())

In [1]:
sm.stop_recording()
sm.stop()

NameError: name 'sm' is not defined